In [1]:
import pandas as pd
from nltk.translate import bleu_score
from jiwer import wer

In [2]:
videos = pd.read_csv("video_list.csv")

In [3]:
videos.head()

,id,runtime,category,url,year,fellow_accuracy_rating,automl_confidence_min,automl_confidence_avg,automl_confidence_max
0,tobacco_rdz99d00,0:01:29,Advertising,https://archive.org/download/tobacco_rdz99d00/...,1966,NaN,NaN,NaN,NaN
1,tobacco_amp91f00,0:05:25,Advertising,https://archive.org/download/tobacco_amp91f00/...,1994,NaN,NaN,NaN,NaN
2,tobacco_xpu03f00,0:11:28,Advertising,https://archive.org/download/tobacco_xpu03f00/...,1961,NaN,NaN,NaN,NaN
3,tobacco_qar62a00,0:12:13,Advertising,https://archive.org/download/tobacco_qar62a00/...,2001,NaN,NaN,NaN,NaN
4,tobacco_ldo23e00,0:55:40,Advertising,https://archive.org/download/tobacco_ldo23e00/...,1957,NaN,NaN,NaN,NaN


In [ ]:
# text cleaning and tokenizing 

# from each text file, create lists of sentence lists and save them with their id name
# remove commas, semicolons, dashes, convert all sentence enders to periods, replace multiple periods with one period
# split by periods and then by spaces

for id in videos.id:
    